In [1]:
import re
import pandas as pd
import time
import pickle
from tqdm import tqdm

## MATH

In [ ]:
from sympy import *
import re

In [ ]:
##sympy setup
!git clone https://github.com/sympy/sympy.git
import os
os.chdir('sympy/')
os.getcwdb()
!python -m pip install -e .

In [ ]:

def equation_solver(text):

  matches = re.findall(r'\[(.*?)\]', text)
  answer=text.split("The answer is ")[-1].replace(".","")
  answer=symbols(answer)
  list_eq=[]
  for i in matches:
    eq=sympify("Eq(" + i.replace("=", ",") + ")")
    list_eq.append(eq)
  result =solve(list_eq)

  if(answer in result.keys()):
    return int(result[answer])
  else:
    return list(result.values())

In [ ]:
df=pd.read_pickle('mistral1_math_test.pkl')

In [ ]:
row=df.iloc[0].to_dict()
text=row['sft_response']
text

In [ ]:
from sympy import *
import re

In [ ]:
equation_solver(text)

In [ ]:
solvable=[]
for i in tqdm(range(df.shape[0])):
  row=df.iloc[i].to_dict()
  text=row['sft_response']
  try:
    ans=equation_solver(text)
    row["solved_answer"]=ans
    solvable.append(row)
  except:
    ans='not solvable'
    print(ans,row)

## WIKI

In [ ]:
!pip install wikipedia
!pip install sentence_transformers

In [ ]:
import re
import pandas as pd
#from datasets import load_dataset
import time
import pickle
from tqdm import tqdm

In [52]:
df=pd.read_pickle("gemma1_wiki_test.pkl")

In [33]:
df.head()

,id,Q,A,type,level,supporting_facts,W,test_prompt,sft_response
0,5a879ab05542996e4f30887e,The Oberoi family is part of a hotel company t...,Delhi,bridge,medium,"{'sent_id': [0, 0], 'title': ['Oberoi family',...",{'sentences': [['The Ritz-Carlton Jakarta is a...,### INSTRUCTION\nYour task is to generate a ch...,C: Search [Oberoi Group head office location -...
1,5a8d7341554299441c6b9fe5,Musician and satirist Allie Goertz wrote a son...,President Richard Nixon,bridge,hard,"{'sent_id': [0, 1, 2, 0], 'title': ['Allie Goe...",{'sentences': [['Lisa Marie Simpson is a ficti...,### INSTRUCTION\nYour task is to generate a ch...,"C: First, find out [who is Milhouse in The Sim..."
2,5a82171f5542990a1d231f4a,What nationality was James Henry Miller's wife?,American,bridge,medium,"{'sent_id': [0, 1, 0], 'title': ['Peggy Seeger...","{'sentences': [['Moloch: or, This Gentile Worl...",### INSTRUCTION\nYour task is to generate a ch...,"C: First, find out [James Henry Miller's wife ..."
3,5a84dd955542997b5ce3ff79,Cadmium Chloride is slightly soluble in this c...,alcohol,bridge,medium,"{'sent_id': [1, 0], 'title': ['Cadmium chlorid...",{'sentences': [['Cadmium chloride is a white c...,### INSTRUCTION\nYour task is to generate a ch...,"C: First, find out [Cadmium Chloride solubilit..."
4,5adf44985542993a75d2646d,Which genus of moth in the world's seventh-lar...,Crambidae,bridge,hard,"{'sent_id': [0, 1, 0, 1], 'title': ['Indogramm...","{'sentences': [['India, officially the Republi...",### INSTRUCTION\nYour task is to generate a ch...,"C: First, find out [the seventh largest countr..."


In [ ]:
import requests
import json
from bs4 import BeautifulSoup
from pprint import pprint
import wikipedia

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')


def wikipedia_seach(query):
  language_code = 'en'

  number_of_results = 3
  headers = {
    # 'Authorization': 'Bearer YOUR_ACCESS_TOKEN',
    'User-Agent': 'YOUR_APP_NAME (YOUR_EMAIL_OR_CONTACT_PAGE)'
  }

  base_url = 'https://api.wikimedia.org/core/v1/wikipedia/'
  endpoint = '/search/page'
  url = base_url + language_code + endpoint

  search_query = query
  parameters = {'q': search_query, 'limit': number_of_results}
  response = requests.get(url, headers=headers, params=parameters)
  response = json.loads(response.text)

  searched_titles=[]
  for page in range(len(response['pages'])):
    display_title = response['pages'][page]['title']
    searched_titles.append(display_title)
    response['pages'][page]["article_url"] = 'http://en.wikipedia.org/?curid=' + str(response['pages'][page]['id'])
    response['pages'][page]['excerpt'] = BeautifulSoup(response['pages'][page]['excerpt']).get_text()

  excerpt_text=search_query+": "+"\n".join([page['excerpt'] for page in response['pages']])
  if(len(searched_titles)!=0):
    #go to page whose title+ excerpt is most similar
    # Encode search query
    query_embedding = sentence_model.encode(search_query)
    page_texts = [page['title'] + ": " + page['excerpt'] for page in response['pages']]
    page_embeddings = sentence_model.encode(page_texts)
    # Calculate similarity scores for each page
    similarities = cosine_similarity([query_embedding], page_embeddings)[0]

    title_score_dict = dict(zip(searched_titles, similarities))

    # Get the index of the page with the highest similarity
    max_similarity_index = similarities.argmax()

    # Output the page id with the highest similarity
    max_similarity_page_id = response['pages'][max_similarity_index]['id']

    try:
      a=wikipedia.page(pageid=max_similarity_page_id)
      full_content=a.content
      return title_score_dict,excerpt_text,full_content
    except:
      return {},"",""
  else:
    return {},"",""

def QA(ques, context):
  result = question_answerer(question=ques,context=context)
  return result['answer'], result['score']

In [5]:
import requests
import json
import numpy as np
from text_to_num import text2num

url = "https://google.serper.dev/search"

def google_search(query):
  payload = json.dumps({
  "q": query
})
  headers = {
  'X-API-KEY': '44218db214e14ddca03d1a19c15af17344e35dcf',
  'Content-Type': 'application/json'
  }
  excerpt_text = ""
  full_content = ""
  most_similar_snippets = ""
  title_score_dict = {}


  try:
    response = requests.request("POST", url, headers=headers, data=payload)
    response = json.loads(response.text)
    #print(response)
    if 'answerBox' in response:
      if 'snippetHighlighted' in response['answerBox']:
        excerpt_text = response['answerBox']['snippetHighlighted'][0]
      elif 'answer' in response['answerBox']:
        excerpt_text = response['answerBox']['answer']
      else:
        excerpt_text = response['answerBox']['snippet']
      
    else:
      #print("HERE")
      full_content = [results['title'] + ": " + results['snippet'] for results in response['organic']]

    if full_content:
      query_embedding = sentence_model.encode(query)
      snippet_embeddings = sentence_model.encode(full_content)
      similarities = cosine_similarity([query_embedding], snippet_embeddings)[0]
      snippets_greater_than_70 = np.where(similarities > 0.50)[0]
      most_similar_snippets = " ".join([full_content[index] for index in snippets_greater_than_70])

    return title_score_dict,excerpt_text,most_similar_snippets
  except:
    return {},"",""

In [6]:
df.head()

,id,Q,A,type,level,supporting_facts,W,test_prompt,sft_response
0,5a879ab05542996e4f30887e,The Oberoi family is part of a hotel company t...,Delhi,bridge,medium,"{'sent_id': [0, 0], 'title': ['Oberoi family',...",{'sentences': [['The Ritz-Carlton Jakarta is a...,### INSTRUCTION\nYour task is to generate a ch...,C: Search [Oberoi Group head office location -...
1,5a8d7341554299441c6b9fe5,Musician and satirist Allie Goertz wrote a son...,President Richard Nixon,bridge,hard,"{'sent_id': [0, 1, 2, 0], 'title': ['Allie Goe...",{'sentences': [['Lisa Marie Simpson is a ficti...,### INSTRUCTION\nYour task is to generate a ch...,"C: First, find out [who is Milhouse in The Sim..."
2,5a82171f5542990a1d231f4a,What nationality was James Henry Miller's wife?,American,bridge,medium,"{'sent_id': [0, 1, 0], 'title': ['Peggy Seeger...","{'sentences': [['Moloch: or, This Gentile Worl...",### INSTRUCTION\nYour task is to generate a ch...,"C: First, find out [James Henry Miller's wife ..."
3,5a84dd955542997b5ce3ff79,Cadmium Chloride is slightly soluble in this c...,alcohol,bridge,medium,"{'sent_id': [1, 0], 'title': ['Cadmium chlorid...",{'sentences': [['Cadmium chloride is a white c...,### INSTRUCTION\nYour task is to generate a ch...,"C: First, find out [Cadmium Chloride solubilit..."
4,5adf44985542993a75d2646d,Which genus of moth in the world's seventh-lar...,Crambidae,bridge,hard,"{'sent_id': [0, 1, 0, 1], 'title': ['Indogramm...","{'sentences': [['India, officially the Republi...",### INSTRUCTION\nYour task is to generate a ch...,"C: First, find out [the seventh largest countr..."


In [34]:
row=2
print(df.iloc[row]['Q'])
print(df.iloc[row]['sft_response'])
print(df.iloc[row]['A'])

 What nationality was James Henry Miller's wife?
C: First, find out [James Henry Miller's wife -Wiki-> y1]. Then determine [y1's nationality -QA(What nationality is y1?)-> y2]. The answer is y2.
American


In [36]:
def process_list_tuple(rep): ## DECISION HERE
  if(str(type(rep[0]))!="<class 'tuple'>"): #if wikipedia article comes as in query, return empty string as it is anyways invalid
    return ""
  #for QA result, best score string returned
  non_empty_rep = [item for item in rep if item]  # Filter out empty tuples
  if non_empty_rep:
    rep = max(non_empty_rep, key=lambda x: x[1])[0]  # Get the string with the highest score
  else:
    rep = ""  # If all tuples are empty, return an empty string
  return str(rep)

def get_variables(text):
  matches = re.findall(r'\[(.*?)\]', text)
  variables={}
  wiki_searches=[]
  errors=[]
  variables['all_eqs']=[]

  def replace_variable(match):
    #print(variables)
    rep = variables.get(match.group(), match.group())
    if isinstance(rep, list):
        rep=process_list_tuple(rep)
    return rep

  for instruction in matches:
    #print('at ', instruction)
    #print(variables)
    #---WIKI TOOL---
    if(instruction.find("-Wiki")!=-1):
      query,ans=instruction.split(" -Wiki-> ") #for in query variables
      in_query_var=re.search(r'y\d+', query)
      if(in_query_var):
        query=re.sub(r'y\d+', replace_variable, query)

      title_score_dict,excerpt_text,full_content=google_search(query)
      #forcing wiki here
      #excerpt_text=''
      #full_content=''
      if not excerpt_text and not full_content:
        print('WIKI SEARCH')
        title_score_dict,excerpt_text,full_content=wikipedia_seach(query)#wikipedia search
        if title_score_dict: #SAVING IN VARIABLES HERE
          #print('HERE updating varibale of wiki answer',ans.strip())
          variables[ans.strip()] = [excerpt_text, full_content]#replaces the answer variable with extracted content of wiki
          wiki_searches.append(title_score_dict)
        else:
          errors.append(f"Error: Wikipedia search unsuccessful for query: '{query}'")
          break
      else:
        variables[ans.strip()] = [excerpt_text, full_content]#replaces the answer variable with extracted content of google 

    #---QA TOOL---
    elif(instruction.find("-QA")!=-1):
      q,a=instruction.split("->")
      result = re.search(r'\((.*?)\)', q)
      if result:
          question = result.group(1)
      else:
          errors.append("Error in finding question")
          continue

      pre=q.split(" -QA")[0]
      context_exc=""
      full_context=""
      if (pre.find("+")!=-1): #if multiple contexts need to be appended

        vars=pre.split("+")
        for i in vars:
          if(i.strip() in variables.keys()):
            if((str(type(variables[i.strip()][0]))!="<class 'tuple'>")):
              context_exc=variables[i.strip()][0] #if variable came from wiki search, it would be a string else it would be a tuple
              full_context=variables[i.strip()][1]
            else:
              t=process_list_tuple(variables[i.strip()])
              #print('-----t',t)
              context_exc=t #if variable came from wiki search, it would be a string else it would be a tuple
              full_context=t
          else:
            errors.append(f"Error: Context variable '{i.strip()}' not found")
            break

      else: #single context QA
        context_var=pre.strip()
        if context_var in variables.keys():
          if((str(type(variables[context_var][0]))!="<class 'tuple'>")):
            context_exc=variables[context_var][0] #if variable came from wiki search, it would be a string else it would be a tuple
            full_context=variables[context_var][1]
          else:
            t=process_list_tuple(variables[context_var])
            context_exc=t #if variable came from wiki search, it would be a string else it would be a tuple
            full_context=t
        else:
          errors.append(f"Error: Context variable '{context_var}' not found")
          break

      #cant send empty context to QA model
      if(context_exc!=""):
        #print(question,context_exc)
        execerpt_run=QA(question, context_exc)
        if(execerpt_run[-1]<0.75 and len(full_context)>0):
          full_run=QA(question, full_context)
        else:
          full_run=()
      else:
        execerpt_run=()
        if(len(full_context)>0):
          full_run=QA(question, full_context)
        else:
          full_run=()


      #print('Updating variable',a.strip())
      variables[a.strip()]=[execerpt_run,full_run]

    #----MATH TOOL----
    else:
      #print('going to math tool')
      print('at math',instruction)
      variables['all_eqs'].append(instruction)
      equation_variables = re.findall(r'y\d+', instruction)
      #print('found varibales',equation_variables)
      for var in equation_variables:
        if(var in variables.keys()):
          #print('on', var)
          var_ans=process_list_tuple(variables[var])
          #print('before process',variables[var])
          #print('on process',var_ans)
          numbers = re.findall(r'\d+(?:\.\d+)?', var_ans)#we try searching for numbers
          if(len(numbers)>0):
            #print('adding equation',var, numbers)
            variables['all_eqs'].append(var+" = "+str(numbers[0])) #making equation for previously retrieved numerical values ### DECISON HERE
            errors.append(f"WARNING: '{str(numbers)}'numbers in var ans")
          else:
            try:
              #var_ans=str(text2num(var_ans, "en"))
              var_ans=str(w2n.word_to_num(var_ans))
              numbers = re.findall(r'\d+(?:\.\d+)?', var_ans)#we try searching for numbers
              if(len(numbers)>0):
                #print('adding equation',var, numbers)
                variables['all_eqs'].append(var+" = "+str(numbers[0])) #making equation for previously retrieved numerical values ### DECISON HERE
                errors.append(f"WARNING: '{str(numbers)}'numbers in var ans")
            except:
              print('var ans is not a numeric',var_ans)
              errors.append(f"Error: '{var_ans}' not numeric")


  return variables,wiki_searches,errors


In [37]:
def solve_all(equations):
  ans_dict={}
  list_eq=[]
  for i in equations:
    eq=sympify("Eq(" + i.replace("=", ",") + ")")
    list_eq.append(eq)
  #print(list_eq)
  result =solve(list_eq)
  #print(result)
  if(str(type(result))=="<class 'list'>"):
    result=result[0]
  for i in result.keys():
      ans_dict[str(i)]=result[i]
  return ans_dict

In [53]:
row=0
print(df.iloc[row]['Q'])
print(df.iloc[row]['sft_response'])
print(df.iloc[row]['A'])
text=df.iloc[row]['sft_response']
matches = re.findall(r'\[(.*?)\]', text)
matches

The Oberoi family is part of a hotel company that has a head office in what city?
C: Search [Oberoi Group head office city -Wiki-> y1]. The city is [y1 -QA(What is the city of the Oberoi Group head office?)-> y2]. The answer is y2.
So, the chain of abstractions for this question would be:
Delhi


['Oberoi Group head office city -Wiki-> y1',
 'y1 -QA(What is the city of the Oberoi Group head office?)-> y2']

In [19]:
v,w,e=get_variables(text)

In [39]:
processed=[]

In [55]:

for row in tqdm(range(33,df.shape[0])):
  row_d=df.iloc[row].to_dict()
  try:
    answer_var=row_d['sft_response'].lower().split("answer is")[-1].replace(".","").strip()
    variables, wiki_searches, errors = get_variables(row_d['sft_response'])
    if(len(variables['all_eqs'])>0):
      #solve eqs
      try:
        variables.update(solve_all(variables['all_eqs']))
      except:
        errors.append("Equaltions not solvable")
    if(answer_var in variables.keys()):
      answer=variables[answer_var]
    else:
      answer="not found"
  except:
    variables=[]
    wiki_searches=[]
    errors=['Outer Error Handling']
    answer="not found"

  row_d['Variables'] = variables
  row_d['Searches'] = wiki_searches
  row_d['Errors'] = errors
  row_d['Generated_answer'] = answer
  processed.append(row_d)

  # Save to pickle file every 50 rows
  if row % 50 == 0 and row != 0:
      processed_df = pd.DataFrame(processed)
      #processed_df.to_pickle(f'processed_rows_wiki2.pkl')


# Save any remaining processed rows to a pickle file
if(processed):
    processed_df = pd.DataFrame(processed)
    #processed_df.to_pickle(f'processed_rows_final_wiki2.pkl')

  1%|▏         | 1/67 [00:01<01:17,  1.17s/it]

WIKI SEARCH


 18%|█▊        | 12/67 [00:56<05:10,  5.65s/it]

at math y1, y2, y3, y4


 42%|████▏     | 28/67 [02:23<02:47,  4.30s/it]

...

 64%|██████▍   | 43/67 [02:50<00:20,  1.15it/s]

WIKI SEARCH


 73%|███████▎  | 49/67 [02:57<00:19,  1.07s/it]

at math y2


100%|██████████| 67/67 [03:19<00:00,  2.98s/it]


In [56]:
processed_df.head()

,id,Q,A,type,level,supporting_facts,W,test_prompt,sft_response,Variables,Searches,Errors,Generated_answer
0,5a879ab05542996e4f30887e,The Oberoi family is part of a hotel company t...,Delhi,bridge,medium,"{'sent_id': [0, 0], 'title': ['Oberoi family',...",{'sentences': [['The Ritz-Carlton Jakarta is a...,### INSTRUCTION\nYour task is to generate a ch...,C: Search [Oberoi Group head office location -...,"{'all_eqs': [], 'y1': ['', 'The Oberoi Group: ...",[],[],"[(), (New Delhi, India, 0.8095722794532776)]"
1,5a8d7341554299441c6b9fe5,Musician and satirist Allie Goertz wrote a son...,President Richard Nixon,bridge,hard,"{'sent_id': [0, 1, 2, 0], 'title': ['Allie Goe...",{'sentences': [['Lisa Marie Simpson is a ficti...,### INSTRUCTION\nYour task is to generate a ch...,"C: First, find out [who is Milhouse in The Sim...","{'all_eqs': [], 'y1': ['Bart Simpson's best fr...",[],[],"[(Bart Simpson, 0.9865113496780396), ()]"
2,5a82171f5542990a1d231f4a,What nationality was James Henry Miller's wife?,American,bridge,medium,"{'sent_id': [0, 1, 0], 'title': ['Peggy Seeger...","{'sentences': [['Moloch: or, This Gentile Worl...",### INSTRUCTION\nYour task is to generate a ch...,"C: First, find out [James Henry Miller's wife ...","{'all_eqs': [], 'y1': ['', 'Henry Miller - Wik...",[],[Error: Context variable 'y1's nationality' no...,not found
3,5a84dd955542997b5ce3ff79,Cadmium Chloride is slightly soluble in this c...,alcohol,bridge,medium,"{'sent_id': [1, 0], 'title': ['Cadmium chlorid...",{'sentences': [['Cadmium chloride is a white c...,### INSTRUCTION\nYour task is to generate a ch...,"C: First, find out [Cadmium Chloride solubilit...","{'all_eqs': [], 'y1': ['soluble in water (133 ...",[],[],"[, Solubility Rules - Chemistry LibreTexts: Th..."
4,5adf44985542993a75d2646d,Which genus of moth in the world's seventh-lar...,Crambidae,bridge,hard,"{'sent_id': [0, 1, 0, 1], 'title': ['Indogramm...","{'sentences': [['India, officially the Republi...",### INSTRUCTION\nYour task is to generate a ch...,"C: First, find out [the seventh largest countr...","{'all_eqs': [], 'y1': ['India', ''], 'y2': ['A...",[],[],"[Aenigmatinea glatzella, ]"


In [57]:
import pandas as pd
from fuzzywuzzy import fuzz
from sentence_transformers import SentenceTransformer, util
import torch
from torch import nn

# Define the additional functions for similarity checks
def exact_match(text1, text2):
    return text1 == text2

def fuzzy_match(text1, text2):
    ratio = fuzz.partial_ratio(text1.lower(), text2.lower())
    return ratio

def semantic_similarity(text1, text2):
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings1 = model.encode(text1, convert_to_tensor=True)
    embeddings2 = model.encode(text2, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    return cosine_scores.item()

def semantic_similarity_phrase_bert(text1, text2):
    phrase_list = [text1, text2]
    model = SentenceTransformer('whaleloops/phrase-bert')
    phrase_embs = model.encode(phrase_list)
    [p1, p2] = phrase_embs
    cos_sim = nn.CosineSimilarity(dim=0)
    return cos_sim(torch.tensor(p1), torch.tensor(p2))

In [58]:
# Update the check_answer function to include new similarity checks
def check_answer(row):
    gen = row['Generated_answer']
    act = row['A']
    all_vars = row['Variables']

    if exact_match(gen, act):
        return True
    if str(gen).find(str(act)) != -1:
        return True
    if str(gen) == str(act):
        return True
    if fuzzy_match(str(gen), act) > 80:  # You can adjust the threshold for fuzzy matching
        return True
    if semantic_similarity(str(gen), act) > 0.6:  # Adjust the threshold for semantic similarity
        return True
    if semantic_similarity_phrase_bert(str(gen), act) > 0.6:  # Adjust the threshold for phrase BERT similarity
        return True
    return False


In [59]:
from tqdm import tqdm

# Assuming 'processed_df' is your DataFrame
tqdm.pandas()  # Initialize tqdm with pandas

# Apply the check_answer function with a progress bar
processed_df['is_final_in_solved'] = processed_df.progress_apply(check_answer, axis=1)

config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 279kB/s]
README.md: 100%|██████████| 10.7k/10.7k [00:00<00:00, 4.85MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 91.5kB/s]
config.json: 100%|██████████| 612/612 [00:00<00:00, 381kB/s]
model.safetensors: 100%|██████████| 90.9M/90.9M [00:01<00:00, 59.2MB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 187kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 17.3MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 9.40MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 290kB/s]
README.md: 100%|██████████| 5.41k/5.41k [00:00<00:00, 2.78MB/s]
sentence_bert_config.json: 100%|██████████| 52.0/52.0 [00:00<00:00, 101kB/s]
config.json: 100%|██████████| 670/670 [00:00<00:00, 1.30MB/s]
pytorch_model.bin: 100%|██████████| 438M/438M [00:07<00:00, 55.4MB/s]
tokenizer_config.json: 100%|██████████| 632/632 [00:00<00:00, 353kB/s]
vocab.txt: 100%

In [60]:
processed_df['is_final_in_solved'].unique()

array([ True, False])

In [61]:
# Count the rows where final_answer is present in solved_answer list
count_valid = processed_df[processed_df['is_final_in_solved']].shape[0]

In [62]:
count_valid

38

In [50]:
processed_df.to_csv("WIKI_GEMMA_result_big_model.csv", index=False)